In [38]:
import sys
sys.path.append("C:/Users/luisalexander/Desktop/3BIO-Scientometrics/SCRIPTS/Librerias")
import pandas as pd 
import numpy as np 
import plotly.express as px
import urllib.request as urllib2
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, SoupStrainer
from bs4 import BeautifulSoup
import urllib.request
import json
import pandas as pd 
import numpy as np
import itertools
import asyncio
from async_class import AsyncClass
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [16]:
# Links de los grupos de investigacion por programa 
links_programa = pd.read_csv('C:/Users/luisalexander/Desktop/3BIO-Scientometrics/DATA/DATA_RAW/links_grupos.csv', encoding='utf-8')

In [17]:
links_programa

,Tipo_programa,URL,Cantidad
0,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...,839
1,"Ciencia, Tecnologia e Innovacion en Ciencias H...",https://scienti.minciencias.gov.co/ciencia-war...,2923
2,Ciencias Basicas,https://scienti.minciencias.gov.co/ciencia-war...,1105
3,"Ciencia, Tecnologia e Innovacion en Salud",https://scienti.minciencias.gov.co/ciencia-war...,1290
4,Biotecnologia,https://scienti.minciencias.gov.co/ciencia-war...,287
5,Ciencias del Mar y los Recursos Hidrobiologicos,https://scienti.minciencias.gov.co/ciencia-war...,87
6,"Ciencia, Tecnologia e Innovacion en Geociencias",https://scienti.minciencias.gov.co/ciencia-war...,65
7,"Ciencia, Tecnologia e Innovacion en Seguridad ...",https://scienti.minciencias.gov.co/ciencia-war...,89
8,"Ciencia, Tecnologia e Innovacion en Tecnologia...",https://scienti.minciencias.gov.co/ciencia-war...,617
9,"Ciencia, Tecnologia e Innovacion en Ingenieria",https://scienti.minciencias.gov.co/ciencia-war...,1102


In [32]:
programa = []
url      = []
for index, row in links_programa.iterrows():
    cantidad  = row['Cantidad']
    Programa  = row['Tipo_programa']
    URL       = row['URL']
    Npaginas  = round(cantidad // 100) + 1
    url      += [URL.replace('PAGINA', str(i+1)).strip() for i in range(Npaginas)]
    programa += [Programa.strip() for i in range(Npaginas)]

In [39]:
links_grupos = pd.DataFrame({'programa':programa, 'url':url})


In [152]:
links_grupos.iloc[0:11,:]

,programa,url
0,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
1,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
2,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
3,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
4,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
5,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
6,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
7,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
8,"Ciencia, Tecnologia e innovacion en Ambiente, ...",https://scienti.minciencias.gov.co/ciencia-war...
9,"Ciencia, Tecnologia e Innovacion en Ciencias H...",https://scienti.minciencias.gov.co/ciencia-war...


In [136]:
async def informacion_basica_grupos(url : str, programa : str): 
    async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                assert response.status == 200
                response  = await response.read()
                soup      = BeautifulSoup(response, 'html.parser')

                link_grupos   = []
                link_perfiles = []
                for link in soup.find_all('a'):
                    temp_link = link.get('href')
                    if '/gruplac/jsp/visualiza/' in temp_link:
                        link_grupos.append(temp_link)
                    if 'jsp/Medicion/graficas/verPerfiles' in temp_link: 
                        link_perfiles.append(temp_link)
    #info = [{'Programa':programa, 'url':val2, 'url_perfiles':val3} for val2, val3 in zip(link_grupos, link_perfiles)]
    val = pd.read_html(response)
    df  = val[2]
    df  = df[df.columns[0][0]][['Código del grupo',	'Nombre grupo',	'Líder', 'Avalado', 'Estado', 'Clasificado en']].iloc[:-1,:]
    df['urls']          = link_grupos
    df['urls_perfiles'] = link_perfiles
    df['programa']      = programa
    return(df)

In [163]:
urls      = links_grupos['url']
programas = links_grupos['programa']
inicio    = 0
fin       = 10
val = await asyncio.gather(*[informacion_basica_grupos(url, programa) for url, programa in zip(urls[inicio:fin], programas[inicio:fin])])

In [170]:
inicio = 0
skip   = 10 
fin    = inicio + skip 
N        = len(urls)            
results  = []
errores  = []
i = 0

while inicio < N: 
    print(i, inicio, fin, '--------------------------')
    val = await asyncio.gather(*[informacion_basica_grupos(url, programa) for url, programa in zip(urls[inicio:fin], programas[inicio:fin])])
    results += val
    if (N - fin) < skip:
        skip = N - fin

    inicio = fin
    fin    = inicio + skip
    i += 1

0 0 10 --------------------------
1 10 20 --------------------------
2 20 30 --------------------------
3 30 40 --------------------------
4 40 50 --------------------------
5 50 60 --------------------------
6 60 70 --------------------------
7 70 80 --------------------------
8 80 90 --------------------------
9 90 100 --------------------------
10 100 110 --------------------------
11 110 120 --------------------------
12 120 124 --------------------------


In [172]:
info_grupos = pd.concat(results)

In [176]:
info_grupos

,Código del grupo,Nombre grupo,Líder,Avalado,Estado,Clasificado en,urls,urls_perfiles,programa
0,COL0044439,Biología de las Criptógamas de Colombia,JAIME AGUIRRE CEBALLOS,2 de 3,Categoría C,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,"Ciencia, Tecnologia e innovacion en Ambiente, ..."
1,COL0000864,Biodiversidad y conservación,JESUS ORLANDO RANGEL CHURIO,2 de 2,Categoría A,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,"Ciencia, Tecnologia e innovacion en Ambiente, ..."
2,COL0048349,Grupo de Investigación en Ecología y Silvicult...,FLAVIO HUMBERTO MORENO HURTADO,1 de 1,Categoría B,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,"Ciencia, Tecnologia e innovacion en Ambiente, ..."
3,COL0136011,RESTAURACIÓN ECOLÓGICA DE TIERRAS DEGRADADAS E...,JUAN DIEGO LEON PELAEZ,1 de 5,Categoría A,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,"Ciencia, Tecnologia e innovacion en Ambiente, ..."
4,COL0026898,Grupo de Investigación en Biocombustibles y Bi...,JORGE ENRIQUE LOPEZ GALAN,3 de 5,Categoría C,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,"Ciencia, Tecnologia e innovacion en Ambiente, ..."
...,...,...,...,...,...,...,...,...,...
6,COL0032634,Grupo de Investigación en Energía y Medio Ambi...,MANUEL DEL JESUS MARTINEZ,2 de 3,Categoría A,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,Investigaciones en Energia y Mineria
7,COL0005501,"GESTIÓN EFICIENTE DE ENERGÍA, Kaí",JUAN CARLOS CAMPOS AVELLA,3 de 4,Categoría A1,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,Investigaciones en Energia y Mineria
8,COL0172054,DSP-ASIC BUILDER GROUP,NaN,1 de 1,Categoría C,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,Investigaciones en Energia y Mineria
9,COL0202359,Grupo de Investigación en Ciencias de la Tierr...,NaN,1 de 1,Categoría 00,Convocatoria 894 de 2021,https://scienti.minciencias.gov.co/gruplac/jsp...,https://scienti.minciencias.gov.co/gruplac/jsp...,Investigaciones en Energia y Mineria
